In [1]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import os
import csv
import re
import pandas as pd
from datetime import datetime

In [7]:
# Function to read the study date from the first DICOM file in a directory
def get_study_date(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".DCM"):
            ds = pydicom.dcmread(os.path.join(directory, filename),stop_before_pixels=True)
            return ds.StudyDate
    return None

# Function to convert DICOM date to day/month/year format
def format_date(dicom_date):
    date = datetime.strptime(dicom_date, "%Y%m%d")
    return date.strftime("%m/%d/%Y")


In [8]:
rootcsv = '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/MatchedCTWithDate.csv'
df = pd.read_csv(rootcsv)
df

,ID,CT,StudyDate
0,4879,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
1,10539,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
2,13194,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
3,14504,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
4,17420,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
...,...,...,...
1022,8029120,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
1023,4766491,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
1024,2214482,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN
1025,2243832,//zkh/appdata/RTDicom/Projectline_modelling_lu...,NaN


In [9]:
ListWithCTDates = []
count = 0

for currCT in df['CT']:
    dicom_date_list = []
    if re.search(r", '",currCT):
        currCT_split = re.split(r", '",currCT)
        CleanList = []
        for i in range(len(currCT_split)):
            CleanList.append(os.path.normpath(currCT_split[i].replace('\\\\', '/')[:-1]))
        uniqueuniqueCleanList = list(set(CleanList))
        for l in range(len(uniqueuniqueCleanList)):
            dicom_date_list.append(get_study_date(uniqueuniqueCleanList[l]))
    else:
        uniqueuniqueCleanList = [currCT.replace('\\\\', '/')]
        dicom_date_list.append(get_study_date(uniqueuniqueCleanList[0]))
        
        
    for m in range(len(dicom_date_list)):
        formatted_date = format_date(dicom_date_list[m])
        ListWithCTDates.append([str(df.loc[count]['ID']),str(uniqueuniqueCleanList[m]),formatted_date])

    count+=1



In [10]:
len(ListWithCTDates)

1057

In [11]:
csv_file_path = '//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/MatchedCTWithDate_Filled.csv'
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    for i in range(len(ListWithCTDates)):
        writer.writerow(ListWithCTDates[i])